In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import requests

def scrape_with_selenium(url, save_directory):
    # 设置 Chrome 浏览器
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 无头模式
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    
    # 启动浏览器
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)

    # 等待页面加载（可视需要调整时间）
    time.sleep(5)

    # 查找 PDF 文件链接
    links = driver.find_elements(By.TAG_NAME, "a")
    pdf_links = [link.get_attribute('href') for link in links if link.get_attribute('href') and 'pdf' in link.get_attribute('href')]

    driver.quit()  # 关闭浏览器

    # 下载 PDF 文件
    if pdf_links:
        os.makedirs(save_directory, exist_ok=True)
        for pdf_url in pdf_links:
            file_name = pdf_url.split("/")[-1]
            save_path = os.path.join(save_directory, file_name)
            
            print(f"正在下载文件: {pdf_url}")
            response = requests.get(pdf_url, stream=True)
            with open(save_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):
                    file.write(chunk)
            print(f"文件已保存到: {save_path}")
    else:
        print("未找到任何 PDF 文件链接。")

# 示例 URL 和保存路径
study_url = "https://clinicaltrials.gov/study/NCT02658019"
save_dir = "./downloads"

scrape_with_selenium(study_url, save_dir)


正在下载文件: https://cdn.clinicaltrials.gov/large-docs/19/NCT02658019/Prot_SAP_000.pdf
文件已保存到: ./downloads/Prot_SAP_000.pdf
